# Pipelines do SkLearn

Hoje vamos aprender uma ferramenta poderosíssima do `sklearn`: _pipelines_ (que em tradução literal é _oleoduto_, mas eu prefiro algo como _linha de montagem_). 

As pipelines nos permitem colocar em sequência todos o passos do nosso projeto de Machine Learning e também automatizar o pré-processamento, treinamento e afinação (_tuning_) de hiperparâmetros. Além disso, elas nos permitem fazer um `GridSearch` não só nos hyperparâmetros de um determinado modelo, mas também nos parâmetros que usamos no pré-processamento.

- _Será que eu preencho os valores nulos com a média ou mediana?_
- _Será que eu uso ou não essa determinada feature?_

Essas entre outras perguntas são muito comuns quando estamos lidando com um projeto. As pipelines permitem que a gente ache exatamente qual é o melhor score que nossos modelos podem ter lidando com essas perguntas através de parâmetros que passamos para um `GridSearch`.

# Importantando bibliotecas e dados

Primeiro, como sempre, vamos importar as bibliotecas e dados que vamos utilizar.

Esse tutorial foi inspirado no segundo capítulo do livro _Hands-On Machine Learning_, o notebook do capítulo pode ser encontrado no [GitHub](https://github.com/ageron/handson-ml2). Então vamos importar o data set utilizado por ele.

In [ ]:
import pandas as pd
import numpy as np

Contextualizando, temos dados de casas no estado da California e seus preços, que é o que queremos predizer.

In [ ]:
input_path = '../input/california-housing-prices/housing.csv'
df = pd.read_csv(input_path)
df.sample(7)

Vemos que existem alguns valores nulos na coluna `total_bedrooms`. 

In [ ]:
df.info()

# Pré-processamento sem pipelines

Antes de ver como usar pipelines, vamos demonstrar como faríamos o pré-processamento dos dados sem elas.
Os passos do pré-processamento serão:
- Preencher os valores nulos
- Normalizar e padronizar os valores numéricos
- Codificar os valores categóricos utilizando One-Hot Encoding

In [ ]:
from sklearn.impute import SimpleImputer

def fill_na(df, strategy='median'):

    num_values = list(df.columns[:-1])

    imputer = SimpleImputer(strategy=strategy)
    imputer.fit(df[num_values])
    df[num_values] = imputer.transform(df[num_values])
    return df


df = fill_na(df)
df.info()

In [ ]:
from sklearn.preprocessing import StandardScaler

def scale(df):
    num_values = list(df.columns[:-1])

    scaler = StandardScaler()
    scaler.fit(df[num_values])
    df[num_values] = scaler.transform(df[num_values])
    return df


df = scale(df)
df.sample(5)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

def encode(df):
    cat_values = ['ocean_proximity']
    
    encoder = OneHotEncoder()
    encoder.fit(df[cat_values])
    columns = [cat_values[0] + '_' + cat_name for cat_name in encoder.categories_][0]
    encoded = pd.DataFrame(encoder.transform(df[cat_values]).toarray(), columns=columns)
    return pd.concat([df, encoded.astype(int)], axis=1).drop('ocean_proximity', axis=1)


df = encode(df)
df

Então, basicamente criei três funções, uma para cada tarefa e vamos modificando o `DataFrame` original até ele se tornar a matriz `X` que vamos passar para nosso modelo de Machine Learning (usando `.fit`).

# Pipelines

Agora vejamos como fazer o mesmo processo, mas utilizando Pipelines. 

Para criar um `Pipeline`, importamos esse objeto e o instanciamos passando uma lista de tuplas. 
- O primeiro valor da tupla é o nome daquele passo na nossa linha de montagem;
- O segundo valor é um `Transformer` do `sklearn`. Ou seja, deve ser um objeto que possua os métodos `transform` e `fit` (pelo menos).

Vamos falar mais em detalhes de `Transformers` mais para frente.

**_Nota_**: o mais correto é dizer que todos os passos devem ser `Transformers`, exceto o último, que pode ser um `Estimator` (ou seja, possuir o método `predict`).

Após criar esse `Pipeline`, podemos chamar os seus métodos `fit` e `transform`. Basicamente, o que ele faz é chamar em sequência cada `Transformer` passando para o próximo a saída retornada pelo anterior. 

In [ ]:
from sklearn.pipeline import Pipeline

num_pipeline = Pipeline([
     ('fillna', SimpleImputer(strategy='median')),
     ('scaler', StandardScaler()),
])

num_values = list(df.columns[:9])
num_pipeline.fit(df[num_values])
num_df_transformed = num_pipeline.transform(df[num_values])
num_df_transformed

Compare com o `DataFrame` que geramos pelo pré-processamento anteriormente e você verá que o resultado é o mesmo. A diferença é que aqui temos diretamente um `ndarray` do `numpy` ao invés de um `DataFrame` do `pandas` (na prática, não muda muito, pois o método `fit` de um estimador transforma internamente `DataFrames` em `ndarrays`).

# Transformações diferentes para colunas diferentes

Bem, agora precisamos fazer o One-Hot Encoding na coluna `ocean_proximity` e juntar com essa matriz que o `Pipeline` gerou, certo?

Infelizmente, apenas com `Pipelines` isso não é possível, pois o `Pipeline` não considera as diferentes colunas de uma matriz. Ele apenas aplica as transformações (é por isso que na hora de dar fit e transform, eu chamo `df[num_values]` ao invés de `df` inteiro).

Entretanto, o objeto `ColumnTransformer` nos permite fazer exatamente o que precisamos: transformar um conjunto de colunas de um jeito e um outro conjunto de colunas de outro e juntar essas colunas em uma única matriz.

Eles funcionam muito similar a `Pipelines`, a diferença é que a tupla recebe um valor a mais: as colunas em que aquela transformação será aplicada.

In [ ]:
from sklearn.compose import ColumnTransformer

cat_values = ['ocean_proximity']

full_pipeline = ColumnTransformer([
    ('numeric', num_pipeline, num_values), 
    ('categorical', OneHotEncoder(), cat_values)
])

df = pd.read_csv(input_path)
X = full_pipeline.fit_transform(df)
X

Note que o objeto `Pipeline` também é um `Transformer` (pois tem os métodos `fit` e `transform`), então podemos considerá-lo como se fosse uma caixa preta e passar para nosso `ColumnTransformer`.

E é isso, temos exatamente a mesma coisa que tínhamos feito antes, mas de uma forma muuito mais prática (claro, entender como `Pipelines` funcionam talvez não seja exatamente fácil, mas com certeza recompensa muito).

Esse tutorial poderia parar por aqui. Mas vamos ver algumas outras coisinhas que podemos fazer utilizando `Pipelines`.

# Criando seus próprios Transformers

O sklearn nos fornece vários transformadores nativos, mas podemos querer criar outros. Um exemplo muito prático é fazer um `Transformer` que gere novas features do seu processo de Feature Engineering. Outro exemplo seria um `Transformer` que matém apenas determinadas features do Data Set (Feature Selection).

Vamos mostrar uma mistura desses dois exemplos. Vou criar um `Transformer` que cria duas novas features e tem um parâmetro booleano se devemos criar ou não uma terceira nova feature (assim poderemos testar mais para frente se incluir ou não incluir essa feature é melhor ou não). 

Para criar um `Transformer`, basicamente basta criar uma classe que implemente os métodos `fit` e `transform`, como já falei (sim, não precisa herdar de nenhuma classe do `sklearn` com o nome `Transformer` ou coisa do tipo. Mostrei isso no Apêndice A do notebook).

Entretanto, existem duas classes que costumamos herdar, pois elas nos ajudam:
- `TransformerMixin` cria para nós um método `fit_transform` automaticamente usando nossos método `fit` e `transform`; e
- `BaseEstimator` cria para nós os métodos `set_params` e `get_params` que são utilizados pelo `GridSearchCV` internamente.

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class CreateNewFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room=True):
        self.rooms_ix = 3
        self.bedrooms_ix = 4
        self.population_ix = 5
        self.households_ix = 6
        self.add_bedrooms_per_room = add_bedrooms_per_room
    
    def fit(self, X, y=None):
        return self # não fazemos nada
    
    def transform(self, X):
        rooms_per_household = X[:, self.rooms_ix] / X[:, self.households_ix]
        population_per_household = X[:, self.population_ix] / X[:, self.households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, self.bedrooms_ix] / X[:, self.rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        return np.c_[X, rooms_per_household, population_per_household]

new_features = CreateNewFeatures(add_bedrooms_per_room=True)
housing_extra_features = new_features.transform(df.values)
housing_extra_features

E pronto, temos um `Transformer` feito por nós mesmos. O limite do que se pode fazer é basicamente definido pela nossa imaginação =)

Agora podemos utilizar esses `Transformers` na nossa `Pipeline` de antes para deixá-la mais sofisticada ainda.

In [ ]:
num_pipeline = Pipeline([
     ('fillna', SimpleImputer(strategy='median')),
     ('feature_creator', CreateNewFeatures(add_bedrooms_per_room=True)),
     ('scaler', StandardScaler()),
])


df_transformed = num_pipeline.fit_transform(df[num_values])
df_transformed

In [ ]:
preprocessing_pipeline = ColumnTransformer([
    ('numeric', num_pipeline, num_values),
    ('categorical', OneHotEncoder(), cat_values)
])

df = pd.read_csv(input_path)
X = preprocessing_pipeline.fit_transform(df)
X

E _voilà_, temos novamente uma matriz prontinha para qualquer modelo de Machine Learning utilizar.

# Pipelines e Hyperparameter Tuning

Por fim, uma das funcionalidade que eu mais acho incrível dos `Pipelines` é a capacidade usá-los junto do `GridSearchCV` (ou `RandomizedSearchCV`) e podermos passar parâmetros dos transformadores em si para serem testados.

Vamos dar uma olhada:

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge

# Dividindo o DataFrame em matrizes X (de features) e y (o target)
# Além disso, definimos quais colunas são numéricas e quais são categóricas (para usar no ColumnTransformer)
target = 'median_house_value'
X = df.loc[:, df.columns != target]
y = df[target]
num_values = np.delete(X.columns, np.where(X.columns == 'ocean_proximity'))
cat_values = ['ocean_proximity']

# Agora definimos nossas Pipelines
num_pipeline = Pipeline([
     ('fillna', SimpleImputer(strategy='median')),
     ('feature_creator', CreateNewFeatures(add_bedrooms_per_room=True)),
     ('scaler', StandardScaler()),
])
preprocessing_pipeline = ColumnTransformer([
    ('numeric', num_pipeline, num_values),
    ('categorical', OneHotEncoder(categories=[df['ocean_proximity'].unique()]), cat_values)
])
final_pipe = Pipeline([
    ('preprocessing', preprocessing_pipeline),
    ('ridge', Ridge())
])

# Veja que a última Pipeline tem dois passos: o pré-processamento e o estimador (o Ridge, que é um modelo linear)

# Agora tem uma parte que pode ser um pouco complexa, mas basta enteder o que está dentro de o que
# Na hora de passar os nomes dos parâmetros para o GridSearch, ele utiliza os nomes que passamos nas tuplas 
# o utiliza dois underlines para se referir a um parâmetro daquele objeto (seria análogo ao ponto que usamos normalmente)
params = {
    'preprocessing__numeric__feature_creator__add_bedrooms_per_room' : [False, True],
    'preprocessing__numeric__fillna__strategy': ['median', 'mean'],
    'ridge__alpha' : [0.1, 1, 10],
}

# De resto, é tudo igual, passamos o estimador (esse Pipeline é um estimador pois o último passo é o Ridge, que é um estimador)
# e passamos os parâmetros, além de outros parâmetros que normalmente usamos em um GridSearch
gs = GridSearchCV(final_pipe, params, cv=5, n_jobs=-1)
gs.fit(X, y)
gs.best_params_ # E temos os melhores parâmetros automaticamente =)

Na minha opinião, a parte mais difícil de entender é esses nomes enormes de parâmetros e entender as abstrações de um `Pipeline` (pensar que o `Pipeline` herda o tipo do objeto no último passo - um `Estimator` ou `Transformer`). Por isso, vou deixar uma imagem para ficar mais claro.

<img src="https://github.com/Giatroo/BeeData_Pipelines-in-Sklearn/blob/main/Arvore-de-Pipelines.jpg?raw=true" alt="drawing" width="600"/>

Além disso, para saber quais são os parâmetros de um `Pipeline`, podemos utilizar o método `get_params`, que nos retorna um dicionário com cada atributo e seu valor padrão. Olhe o Apêndice B para ver o retorno desse método para o nosso `Pipeline` final.

# Apêndice A

In [ ]:
class MyTransformer():
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return np.ones_like(X)
        
df = pd.read_csv(input_path)

my_pipeline = Pipeline([
    ('imputer', SimpleImputer()),
    ('test', MyTransformer()), 
])

my_pipeline.fit_transform(df[num_values])
# Mesmo não herdando de nada, o sklearn reconhece nossa classe como um Transformer =)

# Apêndice B

In [ ]:
final_pipe.get_params()

# Conclusão e Aprofundamento

Por hoje é tudo =)

Espero que você tenha gostado e aproveitado bastante. 

Concluímos que os `Pipelines` são uma ferramenta incrível que podemos utilizar para agilizar muito nossa vida como cientistas de dados. Além disso, aprendemos um pouco sobre como o `sklearn` opera (os diferentes tipos de objetos dele) e como criar nossos próprios `Transformers`, além de aplicar diferentes transformações a diferentes colunas utilizando `ColumnTransformers`.

Para se aprofundar e revisar, recomendo dar uma buscada no livro de onde eu tirei esses exemplos, que já citei, além de dar uma olhada nos seguintes recursos:

[Livro - Hands On Machine Learning](https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/)

[Guia do usuário - Pipeline and composite estimators](https://scikit-learn.org/stable/modules/compose.html#combining-estimators)

[Documentação Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)

[Documentação make_pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html#sklearn.pipeline.make_pipeline)

[Documentação ColumnTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html#sklearn.compose.ColumnTransformer)

[Documentação TransformedTargetRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.compose.TransformedTargetRegressor.html#sklearn.compose.TransformedTargetRegressor)

[Tutorial do towards data science](https://towardsdatascience.com/pipelines-custom-transformers-in-scikit-learn-the-step-by-step-guide-with-python-code-4a7d9b068156)



**~Lucas Paiolla, 23/04/2021**